# Библиотеки

In [1]:
import os
import sys

import gdown
import findspark
import numpy as np
import pandas as pd

from pathlib import Path

# Скачивание транзакций

In [2]:
trans_path = 'spark_transactions.parquet'

if not Path(trans_path).exists():
    gdown.download(id='14qjCQtMItkq1crn00TjVwDzuK6IPpoH_', output=trans_path)

Downloading...
From (original): https://drive.google.com/uc?id=14qjCQtMItkq1crn00TjVwDzuK6IPpoH_
From (redirected): https://drive.google.com/uc?id=14qjCQtMItkq1crn00TjVwDzuK6IPpoH_&confirm=t&uuid=68825388-404d-4496-91b9-e6cf8b09a64a
To: C:\Users\Admin\Desktop\pyspark\lesson_3\spark_transactions.parquet
100%|███████████████████████████████████████████████████████████████████████████████| 177M/177M [01:04<00:00, 2.74MB/s]


# PySpark контекст

Как установить PySpark на Windows - [Полный гид](https://www.machinelearningplus.com/pyspark/install-pyspark-on-windows/)

Добавим `PYSPARK_PYTHON` и `PYSPARK_DRIVER_PYTHON` в PATH. [Подробнее](https://stackoverflow.com/questions/48260412/environment-variables-pyspark-python-and-pyspark-driver-python):

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

Скачаем бинарники `hadoop.dll` и `winutils.exe` последних версий с [гитхаба](https://github.com/kontext-tech/winutils). Оба файла поместим в папку, укажем путь до неё в переменной среды `HADOOP_HOME`. Помимо этого, файл `hadoop.dll` поместим в папку `C:\Windows\System32`. Подробнее [здесь](https://stackoverflow.com/questions/41851066/exception-in-thread-main-java-lang-unsatisfiedlinkerror-org-apache-hadoop-io).

In [4]:
os.environ['HADOOP_HOME'] = 'C:\\hadoop'

In [5]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050')\
                  .set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы

sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()
display(spark)

In [6]:
print(f"JDK version: {os.popen('java --version').read()}")
print(f"Python version: {sys.version}")
print(f"Spark version: {pyspark.__version__}")
print(f"Hadoop version: {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

JDK version: 
Python version: 3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)]
Spark version: 3.5.1
Hadoop version: 3.3.4


# Занятие

## Создание DataFrame

Из RDD

In [7]:
def cleaning(row):
    row = row.split('\t')[:3]
    row = [float(val) for val in row]
    return row

In [8]:
ratings = sc.textFile('../lesson_1/user_ratedmovies.dat', 20)

first_row = ratings.first()
ratings = ratings.filter(lambda row: row != first_row)\
                 .map(cleaning)

In [9]:
columns = first_row.split('\t')[:3]

In [10]:
columns

['userID', 'movieID', 'rating']

In [11]:
ratings.take(5)

[[75.0, 3.0, 1.0],
 [75.0, 32.0, 4.5],
 [75.0, 110.0, 4.0],
 [75.0, 160.0, 2.0],
 [75.0, 163.0, 4.0]]

In [12]:
df_rdd = spark.createDataFrame(ratings, columns)

In [13]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Можно еще вот так:

In [14]:
df_rdd = ratings.toDF(columns)

In [15]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Так, а если не хочу вот эти приседания с RDD, а хочу сразу из файла?

In [16]:
df = spark.read\
          .format('csv')\
          .options(**{'sep': '\t', 'header': 'true'})\
          .load('../lesson_1/user_ratedmovies.dat')

In [17]:
df

DataFrame[userID: string, movieID: string, rating: string, date_day: string, date_month: string, date_year: string, date_hour: string, date_minute: string, date_second: string]

Все в string, так не пойдет, давайте автоматически определим тип данных

In [18]:
df = spark.read\
          .format('csv')\
          .options(**{'sep': '\t', 'header': 'true', 'inferSchema': 'true'})\
          .load('../lesson_1/user_ratedmovies.dat')

In [19]:
df

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

In [20]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

А можно заранее сказать, какой тип данных я ожидаю?

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [22]:
schema = StructType([ \
    StructField('userID',IntegerType(), True), \
    StructField('movieID',IntegerType(), True), \
    StructField('rating',DoubleType(), True), \
    StructField('date_day', StringType(), True), \
    StructField('date_month', StringType(), True), \
    StructField('date_year', IntegerType(), True), \
    StructField('date_hour', IntegerType(), True), \
    StructField('date_minute', IntegerType(), True), \
    StructField('date_second', IntegerType(), True)
  ])

In [23]:
df = spark.read\
          .format('csv')\
          .options(**{'sep': '\t', 'header': 'true'})\
          .schema(schema)\
          .load('../lesson_1/user_ratedmovies.dat')

In [24]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: string (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



In [25]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

Но есть уже готовая обертка под все нужды

In [26]:
df = spark.read.csv(path='../lesson_1/user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [27]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



## Сохрание DataFrame

Так, а как сохранить? Лучше быть аккуратнее с `overwrite`, перезапишет весь указанный путь, `append` будет безопаснее

In [28]:
df.write.option('header', False)\
        .mode('overwrite')\
        .parquet('write_1.parquet')

А что с партицированием?

In [29]:
df.write.option('header', True)\
        .partitionBy('date_year')\
        .mode('overwrite')\
        .parquet('write_2.parquet')

## Схемы данных

Кстати, раз уж заговорили про схемы данных, то их можно задавать интереснее, например, под группированные данные

In [30]:
structureData = [
    (('James','','Smith'),'36636','M',3100),
    (('Michael','Rose',''),'40288','M',4300),
    (('Robert','','Williams'),'42114','M',1400),
    (('Maria','Anne','Jones'),'39192','F',5500),
    (('Jen','Mary','Brown'),'','F',-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



Со структурой можно работать и менять ее под ваши нужны

In [31]:
updatedDF = df2.withColumn('OtherInfo', 
    F.struct(F.col('id').alias('identifier'),
    F.col('gender').alias('gender'),
    F.col('salary').alias('salary'),
    F.when(F.col('salary').cast(IntegerType()) < 2000,'Low')
      .when(F.col('salary').cast(IntegerType()) < 4000,'Medium')
      .otherwise('High').alias('Salary_Grade')
  )).drop('id','gender','salary')

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- OtherInfo: struct (nullable = false)
 |    |-- identifier: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
|name                |OtherInfo               |
+--------------------+------------------------+
|{James, , Smith}    |{36636, M, 3100, Medium}|
|{Michael, Rose, }   |{40288, M, 4300, High}  |
|{Robert, , Williams}|{42114, M, 1400, Low}   |
|{Maria, Anne, Jones}|{39192, F, 5500, High}  |
|{Jen, Mary, Brown}  |{, F, -1, Low}          |
+--------------------+------------------------+



Что мы там сделали????

1) Создали новую структуру данных OtherInfo

2) Передали туда id (переименовав столбец), gender, salary

3) Создали столбец Salary_grade из условий

4) удалили id, gender, salary из старой структуры

Есть и еще структуры данных!

In [32]:
from pyspark.sql.types import ArrayType, MapType

In [33]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

In [34]:
structureData = [
    (('James','','Smith'), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (('Michael','Rose',''), ['car', 'football'], {2: 'b'}),
    (('Robert','','Williams'), ['box', 'music'], {3: 'c'})
  ]

In [35]:
df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: integer
 |    |-- value: string (valueContainsNull = true)

+--------------------+-----------------+----------------+
|name                |hobbies          |properties      |
+--------------------+-----------------+----------------+
|{James, , Smith}    |[car, volleyball]|{4 -> d, 1 -> a}|
|{Michael, Rose, }   |[car, football]  |{2 -> b}        |
|{Robert, , Williams}|[box, music]     |{3 -> c}        |
+--------------------+-----------------+----------------+



In [36]:
df3.select('properties').collect()

[Row(properties={4: 'd', 1: 'a'}),
 Row(properties={2: 'b'}),
 Row(properties={3: 'c'})]

## Описание данных

Общее описание данных

In [37]:
df = spark.read.csv(path='../lesson_1/user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [38]:
df.describe().toPandas()

,summary,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,count,855598,855598,855598,855598,855598,855598,855598,855598,855598
1,mean,35190.83255103448,8710.179402008887,3.437945156487042,15.568923723524366,6.54079485926802,2005.323717446745,12.124531614145896,29.645382527775894,29.510222090280717
2,stddev,20385.003346992034,14446.852908494124,1.0025608721610393,8.951201361275194,3.506399425908951,2.2826973219014897,7.596701614618094,17.26730947820293,17.310302009988877
3,min,75,1,0.5,1,1,1997,0,0,0
4,max,71534,65133,5.0,31,12,2009,23,59,59


In [39]:
df.summary().toPandas()

,summary,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,count,855598,855598,855598,855598,855598,855598,855598,855598,855598
1,mean,35190.83255103448,8710.179402008887,3.437945156487042,15.568923723524366,6.54079485926802,2005.323717446745,12.124531614145896,29.645382527775894,29.510222090280717
2,stddev,20385.003346992034,14446.852908494124,1.0025608721610393,8.951201361275194,3.506399425908951,2.2826973219014897,7.596701614618094,17.26730947820293,17.310302009988877
3,min,75,1,0.5,1,1,1997,0,0,0
4,25%,18161,1367,3.0,8,4,2004,5,15,15
5,50%,33866,3249,3.5,15,7,2006,13,30,30
6,75%,52004,6534,4.0,23,10,2007,19,45,44
7,max,71534,65133,5.0,31,12,2009,23,59,59


Количество записей

In [40]:
df.count()

855598

Количество партиций

In [41]:
df.rdd.getNumPartitions()

7

 Менять число партиций можно, все как с rdd

In [42]:
df = df.repartition(5)

In [43]:
df.rdd.getNumPartitions()

5

In [44]:
df = df.coalesce(2)

In [45]:
df.rdd.getNumPartitions()

2

## Различные методы

Ну теперь давайте тыкать 

### dropDuplicates

Удаляем дубликаты и помним, что есть actions и transformations, count заставит все сделать

In [46]:
df_without_duplicates = df.drop_duplicates()

Есть alias

In [47]:
df_without_duplicates = df.dropDuplicates()

Как удалить дубликаты по отдельным колонкам?

In [48]:
df_without_duplicates = df.drop_duplicates(['userID', 'rating'])

In [49]:
df_without_duplicates.count()

18847

In [50]:
df_without_duplicates.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  5925|   6711|   4.0|      22|        11|     2006|       20|          2|          0|
|  3512|    593|   4.5|      17|         7|     2008|       19|          6|         32|
| 18718|   8360|   2.0|      17|         9|     2008|       16|         43|         15|
| 19519|   3408|   3.0|      21|         2|     2006|        1|         39|         13|
| 15816|  34162|   4.5|       4|         1|     2007|       23|          1|         27|
| 16862|   8638|   4.5|      10|         2|     2006|        5|         10|          4|
| 11930|   1961|   3.0|       2|         4|     2005|       18|          4|         31|
| 28962|    593|   4.0|       4|        12|     2006|        8|         47|         26|
| 23129|     10|   4.5|       5|

### corr

Корреляции

In [51]:
df.corr('rating', 'date_day')

0.016638388440498076

In [52]:
df.corr('rating', 'date_hour')

-0.012518740192686511

In [53]:
df.corr('rating', 'date_year')

-0.0021299262699295183

### toPandas

Как закинуть данные в любимый pandas?

Самый простой вариант - встроенный метод, но он жутко медленный при существенном объеме данных. Лучше сохранить паркет и считать через `pd.read_parquet()`

In [54]:
import pandas as pd

In [55]:
%%timeit 

pandas_df = df.toPandas()

4.34 s ± 274 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
pandas_df = df.toPandas()
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,6393,2071,3.5,4,1,2006,21,28,23
1,1198,33794,4.0,1,9,2007,14,49,33
2,267,10,3.0,30,1,2006,0,2,17
3,8168,1923,3.5,4,12,2006,16,40,44
4,9522,5292,5.0,8,4,2002,1,50,21
...,...,...,...,...,...,...,...,...,...
855593,70362,7566,4.0,19,11,2005,6,32,9
855594,67068,1208,5.0,30,9,1998,22,59,14
855595,67687,273,4.0,8,4,2007,2,48,46
855596,66892,2966,4.0,28,6,2006,0,17,31


In [57]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int32  
 1   movieID      855598 non-null  int32  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int32  
 4   date_month   855598 non-null  int32  
 5   date_year    855598 non-null  int32  
 6   date_hour    855598 non-null  int32  
 7   date_minute  855598 non-null  int32  
 8   date_second  855598 non-null  int32  
dtypes: float64(1), int32(8)
memory usage: 32.6 MB


In [58]:
%%timeit 

pandas_df = pd.read_parquet('write_1.parquet')

31 ms ± 6.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
pandas_df = pd.read_parquet('write_1.parquet')
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30
...,...,...,...,...,...,...,...,...,...
855593,71534,44555,4.0,3,12,2007,3,5,38
855594,71534,46578,4.0,3,12,2007,2,56,44
855595,71534,48516,4.5,3,12,2007,2,53,46
855596,71534,61075,5.0,10,10,2008,9,56,5


In [60]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int32  
 1   movieID      855598 non-null  int32  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int32  
 4   date_month   855598 non-null  int32  
 5   date_year    855598 non-null  int32  
 6   date_hour    855598 non-null  int32  
 7   date_minute  855598 non-null  int32  
 8   date_second  855598 non-null  int32  
dtypes: float64(1), int32(8)
memory usage: 32.6 MB


Как говорили на лекции, может все упасть например тут (когда делаем toPandas). Как перейти к итератору?

prefetchPartitions - подготавливать ли следующую партию данных, пока не запросили

In [61]:
iter_df = df.toLocalIterator()

In [62]:
row = iter_df.send(None)

In [63]:
row

Row(userID=6393, movieID=2071, rating=3.5, date_day=4, date_month=1, date_year=2006, date_hour=21, date_minute=28, date_second=23)

In [64]:
row.asDict()

{'userID': 6393,
 'movieID': 2071,
 'rating': 3.5,
 'date_day': 4,
 'date_month': 1,
 'date_year': 2006,
 'date_hour': 21,
 'date_minute': 28,
 'date_second': 23}

Отсюда идея: можно вытягивать данные по 1 записи и записывать в датафрейм. Долго, но зато отработает.

In [65]:
iter_df = df.toLocalIterator()

In [66]:
list_of_rows = [value for value in iter_df]
print(len(list_of_rows))

855598


In [67]:
df.columns

['userID',
 'movieID',
 'rating',
 'date_day',
 'date_month',
 'date_year',
 'date_hour',
 'date_minute',
 'date_second']

In [68]:
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [69]:
%%timeit

iter_df = df.toLocalIterator()
list_of_rows = [value for value in iter_df]
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

4.54 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,6393,2071,3.5,4,1,2006,21,28,23
1,1198,33794,4.0,1,9,2007,14,49,33
2,267,10,3.0,30,1,2006,0,2,17
3,8168,1923,3.5,4,12,2006,16,40,44
4,9522,5292,5.0,8,4,2002,1,50,21
...,...,...,...,...,...,...,...,...,...
855593,70362,7566,4.0,19,11,2005,6,32,9
855594,67068,1208,5.0,30,9,1998,22,59,14
855595,67687,273,4.0,8,4,2007,2,48,46
855596,66892,2966,4.0,28,6,2006,0,17,31


In [71]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int64  
 1   movieID      855598 non-null  int64  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int64  
 4   date_month   855598 non-null  int64  
 5   date_year    855598 non-null  int64  
 6   date_hour    855598 non-null  int64  
 7   date_minute  855598 non-null  int64  
 8   date_second  855598 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 58.7 MB


### show

In [72]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|
|  1198|  33794|   4.0|       1|         9|     2007|       14|         49|         33|
|   267|     10|   3.0|      30|         1|     2006|        0|          2|         17|
|  8168|   1923|   3.5|       4|        12|     2006|       16|         40|         44|
|  9522|   5292|   5.0|       8|         4|     2002|        1|         50|         21|
|  2692|    277|   2.0|      16|         7|     2008|       18|         32|         20|
|  1224|    832|   3.5|      17|         5|     2003|        2|          5|         23|
| 10674|   1917|   2.5|      11|        10|     2007|        0|         15|         16|
|  8048|    235|   4.0|      29|

Обрезаем до 2 символов

In [73]:
df.show(10, truncate=2)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    63|     20|    3.|       4|         1|       20|       21|         28|         23|
|    11|     33|    4.|       1|         9|       20|       14|         49|         33|
|    26|     10|    3.|      30|         1|       20|        0|          2|         17|
|    81|     19|    3.|       4|        12|       20|       16|         40|         44|
|    95|     52|    5.|       8|         4|       20|        1|         50|         21|
|    26|     27|    2.|      16|         7|       20|       18|         32|         20|
|    12|     83|    3.|      17|         5|       20|        2|          5|         23|
|    10|     19|    2.|      11|        10|       20|        0|         15|         16|
|    80|     23|    4.|      29|

вертикальное отображение

In [74]:
df.show(10, vertical=True)

-RECORD 0------------
 userID      | 6393  
 movieID     | 2071  
 rating      | 3.5   
 date_day    | 4     
 date_month  | 1     
 date_year   | 2006  
 date_hour   | 21    
 date_minute | 28    
 date_second | 23    
-RECORD 1------------
 userID      | 1198  
 movieID     | 33794 
 rating      | 4.0   
 date_day    | 1     
 date_month  | 9     
 date_year   | 2007  
 date_hour   | 14    
 date_minute | 49    
 date_second | 33    
-RECORD 2------------
 userID      | 267   
 movieID     | 10    
 rating      | 3.0   
 date_day    | 30    
 date_month  | 1     
 date_year   | 2006  
 date_hour   | 0     
 date_minute | 2     
 date_second | 17    
-RECORD 3------------
 userID      | 8168  
 movieID     | 1923  
 rating      | 3.5   
 date_day    | 4     
 date_month  | 12    
 date_year   | 2006  
 date_hour   | 16    
 date_minute | 40    
 date_second | 44    
-RECORD 4------------
 userID      | 9522  
 movieID     | 5292  
 rating      | 5.0   
 date_day    | 8     
 date_mont

### select

В PySpark функция `select()` используется для выбора одного, нескольких столбцов по индексу, всех столбцов из списка и вложенных столбцов из фрейма данных. Функция PySpark `select()` является функцией преобразования, поэтому она возвращает новый фрейм данных с выбранными столбцами.

In [75]:
df.columns

['userID',
 'movieID',
 'rating',
 'date_day',
 'date_month',
 'date_year',
 'date_hour',
 'date_minute',
 'date_second']

Pandas-style тут не приветствуется:

In [76]:
# df.userID.show(5) # 'Column' object is not callable

In [77]:
df.select('userID').show(5)

+------+
|userID|
+------+
| 10231|
| 10231|
| 10231|
| 10231|
| 10231|
+------+
only showing top 5 rows



Куча вариантов, выбирайте любой

In [78]:
df.select('userID', 'rating').show(5)

+------+------+
|userID|rating|
+------+------+
| 11255|   0.5|
|  3057|   0.5|
|  2429|   4.5|
|  2429|   4.5|
|  2429|   4.5|
+------+------+
only showing top 5 rows



In [79]:
df.select(['userID', 'rating']).show(5)

+------+------+
|userID|rating|
+------+------+
| 11255|   0.5|
|  3057|   0.5|
|  2429|   4.5|
|  2429|   4.5|
|  2429|   4.5|
+------+------+
only showing top 5 rows



In [80]:
df.select(df.userID, df.rating).show(5)

+------+------+
|userID|rating|
+------+------+
| 11255|   0.5|
|  3057|   0.5|
|  2429|   4.5|
|  2429|   4.5|
|  2429|   4.5|
+------+------+
only showing top 5 rows



In [81]:
df.select(df['userID'], df['rating']).show(5)

+------+------+
|userID|rating|
+------+------+
| 11255|   0.5|
|  3057|   0.5|
|  2429|   4.5|
|  2429|   4.5|
|  2429|   4.5|
+------+------+
only showing top 5 rows



In [82]:
df.select(F.col('userID'), F.col('rating')).show(5)

+------+------+
|userID|rating|
+------+------+
| 11255|   0.5|
|  3057|   0.5|
|  2429|   4.5|
|  2429|   4.5|
|  2429|   4.5|
+------+------+
only showing top 5 rows



можно налету переименовать столбец

In [83]:
df.select(df.userID, df.rating.alias('mark')).show(5)

+------+----+
|userID|mark|
+------+----+
| 11255| 0.5|
|  3057| 0.5|
|  2429| 4.5|
|  2429| 4.5|
|  2429| 4.5|
+------+----+
only showing top 5 rows



In [84]:
# Регулярки:
df.select(df.colRegex('`d+.*y`')).show(5)

+--------+
|date_day|
+--------+
|      19|
|      19|
|      19|
|      19|
|      19|
+--------+
only showing top 5 rows



примеры с другим датафреймом, где структура сложнее

In [85]:
structureData = [
    (('James', '', 'Smith'), '36636', 'M', 3100),
    (('Michael', 'Rose', ''), '40288', 'M', 4300),
    (('Robert', '', 'Williams'), '42114', 'M', 1400),
    (('Maria', 'Anne', 'Jones'), '39192', 'F', 5500),
    (('Jen', 'Mary', 'Brown'), '', 'F', -1)
]

structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
            ])),
        StructField('id', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('salary', IntegerType(), True)
    ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



In [86]:
df2.select('name').show(5)

+--------------------+
|                name|
+--------------------+
|    {James, , Smith}|
|   {Michael, Rose, }|
|{Robert, , Williams}|
|{Maria, Anne, Jones}|
|  {Jen, Mary, Brown}|
+--------------------+



In [87]:
df2.select('name.lastname').show(5)

+--------+
|lastname|
+--------+
|   Smith|
|        |
|Williams|
|   Jones|
|   Brown|
+--------+



In [88]:
df2.select('name.firstname', 'name.lastname').show(5)

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|        |
|   Robert|Williams|
|    Maria|   Jones|
|      Jen|   Brown|
+---------+--------+



### withColumn

PySpark `withColumn()` - это функция преобразования (transform), которая используется для изменения значения, преобразования типа данных существующего столбца, создания нового столбца и многого другого. Поговорим о часто используемых операциях со столбцами данных PySpark, используя примеры.

In [89]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Меняем тип данных

In [90]:
df.withColumn('date_month', F.col('date_month').cast('String')).printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Модифицировать столбец/создать новый

In [91]:
df.withColumn('rating_x_10', F.col('rating') * 10).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|rating_x_10|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|       35.0|
|  1198|  33794|   4.0|       1|         9|     2007|       14|         49|         33|       40.0|
|   267|     10|   3.0|      30|         1|     2006|        0|          2|         17|       30.0|
|  8168|   1923|   3.5|       4|        12|     2006|       16|         40|         44|       35.0|
|  9522|   5292|   5.0|       8|         4|     2002|        1|         50|         21|       50.0|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
only showing top 5 rows



Делаем 2 константных столбца

In [92]:
df.withColumn('fix_1', F.lit(1)).withColumn('fix_2', F.lit(2)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|fix_1|fix_2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|    1|    2|
|  1198|  33794|   4.0|       1|         9|     2007|       14|         49|         33|    1|    2|
|   267|     10|   3.0|      30|         1|     2006|        0|          2|         17|    1|    2|
|  8168|   1923|   3.5|       4|        12|     2006|       16|         40|         44|    1|    2|
|  9522|   5292|   5.0|       8|         4|     2002|        1|         50|         21|    1|    2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
only showing top 5 rows



### withColumnsRenamed

Предыдущий вариант не давал возможности переименовать столбцы, это можно сделать иначе

In [93]:
df.withColumnRenamed('rating', 'mark').show(5)

+------+-------+----+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|mark|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+----+--------+----------+---------+---------+-----------+-----------+
|  6393|   2071| 3.5|       4|         1|     2006|       21|         28|         23|
|  1198|  33794| 4.0|       1|         9|     2007|       14|         49|         33|
|   267|     10| 3.0|      30|         1|     2006|        0|          2|         17|
|  8168|   1923| 3.5|       4|        12|     2006|       16|         40|         44|
|  9522|   5292| 5.0|       8|         4|     2002|        1|         50|         21|
+------+-------+----+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



### filter & where

Функция PySpark `filter()` используется для фильтрации строк из RDD / DataFrame на основе заданного условия или выражения SQL, вы также можете использовать предложение `where()` вместо `filter()` обе эти функции работают аналогично.

1 условие

In [94]:
df.filter(df.rating == 5.0).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  8114|   4351|   5.0|      13|         6|     2001|       23|         52|         57|
|  8892|    529|   5.0|       2|         7|     2004|       15|         11|         12|
| 10791|   4238|   5.0|       3|         8|     2002|        1|         20|         31|
|  6227|     32|   5.0|       6|         8|     2006|       18|         58|         11|
|  4895|  27728|   5.0|       1|         1|     2006|        4|         15|          2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [95]:
df.filter(~(df.rating == 5.0)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|
|  1198|  33794|   4.0|       1|         9|     2007|       14|         49|         33|
|   267|     10|   3.0|      30|         1|     2006|        0|          2|         17|
|  8168|   1923|   3.5|       4|        12|     2006|       16|         40|         44|
|  7248|  32371|   4.0|      14|         1|     2008|        9|         33|         29|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [96]:
df.filter('rating = 5').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  8114|   4351|   5.0|      13|         6|     2001|       23|         52|         57|
|  8892|    529|   5.0|       2|         7|     2004|       15|         11|         12|
| 10791|   4238|   5.0|       3|         8|     2002|        1|         20|         31|
|  6227|     32|   5.0|       6|         8|     2006|       18|         58|         11|
|  4895|  27728|   5.0|       1|         1|     2006|        4|         15|          2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



Несколько условий

In [97]:
df.filter((df.rating == 5.0) & (df.date_year == 2006)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  6393|    922|   5.0|      26|         2|     2006|       21|         54|         22|
|  8041|   1682|   5.0|       9|         1|     2006|       17|         31|         55|
|  4827|   1193|   5.0|       4|         1|     2006|       22|         12|         37|
|  6323|   2278|   5.0|       3|         1|     2006|        9|          3|          8|
|  6385|  42004|   5.0|       8|        12|     2006|        4|         33|         48|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [98]:
df.filter('(rating = 5.0) and (date_year = 2006)').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  6393|    922|   5.0|      26|         2|     2006|       21|         54|         22|
|  8041|   1682|   5.0|       9|         1|     2006|       17|         31|         55|
|  4827|   1193|   5.0|       4|         1|     2006|       22|         12|         37|
|  6323|   2278|   5.0|       3|         1|     2006|        9|          3|          8|
|  6385|  42004|   5.0|       8|        12|     2006|        4|         33|         48|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [99]:
df.filter('(rating = 5.0) and (userID between 70 and 80)').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    78|   1214|   5.0|       7|         5|     2004|       23|          6|         34|
|    78|     32|   5.0|       7|         5|     2004|       23|         32|         18|
|    78|   2502|   5.0|       7|         5|     2004|       23|         38|         29|
|    78|    750|   5.0|       7|         5|     2004|       23|          3|         45|
|    78|   1232|   5.0|       7|         5|     2004|       23|         29|         49|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



фильтр по списку значений из list

In [100]:
years = [2006, 2007]

In [101]:
df.filter(df.date_year.isin(years)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  4352|  30812|   3.0|       2|         3|     2006|       22|         34|         12|
|  5954|   4007|   4.0|       8|         1|     2006|       22|          5|          2|
|  8168|   1923|   3.5|       4|        12|     2006|       16|         40|         44|
|  6264|   1907|   3.5|       3|        12|     2006|       11|          4|         14|
|  7849|   1209|   5.0|       8|        12|     2007|        1|         57|         22|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



 проверим

In [102]:
df.filter(df.date_year.isin(years))\
  .select('date_year')\
  .dropDuplicates()\
  .collect()

[Row(date_year=2007), Row(date_year=2006)]

создадим игрушечный датайфрейм для текстовых столбцов

In [103]:
data2 = [
    (2,'Michael Rose'),
    (3,'Robert Williams'),
    (4,'Rames Rose'),
    (5,'Rames Black'),
    (6, 'Albus Torch'),
    (7, 'Fred Tf')
]

df2 = spark.createDataFrame(data2, ['id', 'name'])

In [104]:
df2.show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
|  6|    Albus Torch|
|  7|        Fred Tf|
+---+---------------+



In [105]:
df2.filter("name like 'R%'").show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
+---+---------------+



In [106]:
df2.filter(df2.name.startswith('R')).show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
+---+---------------+



In [107]:
df2.filter(df2.name.endswith('Tf')).show()

+---+-------+
| id|   name|
+---+-------+
|  7|Fred Tf|
+---+-------+



In [108]:
df2.filter(df2.name.contains('Wil')).show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
+---+---------------+



Бывает, что у нас внутри датафрейма есть массив и с ним что-то хочется сделать

In [109]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

structureData = [
    (('James','','Smith'), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (('Michael','Rose',''), ['car', 'football'], {2: 'b'}),
    (('Robert','','Williams'), ['box', 'music'], {3: 'c'})
]

df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: integer
 |    |-- value: string (valueContainsNull = true)

+--------------------+-----------------+----------------+
|name                |hobbies          |properties      |
+--------------------+-----------------+----------------+
|{James, , Smith}    |[car, volleyball]|{4 -> d, 1 -> a}|
|{Michael, Rose, }   |[car, football]  |{2 -> b}        |
|{Robert, , Williams}|[box, music]     |{3 -> c}        |
+--------------------+-----------------+----------------+



In [110]:
df3.filter(F.array_contains(df3.hobbies, 'football'))\
   .show()  

+-----------------+---------------+----------+
|             name|        hobbies|properties|
+-----------------+---------------+----------+
|{Michael, Rose, }|[car, football]|  {2 -> b}|
+-----------------+---------------+----------+



### orderBy

сделаем еще фильтрацию, чтобы увидеть результат (`orderBy` тут аналог)

In [111]:
df.filter(df.userID == 75).sort(df.date_minute, df.rating.desc()).show(20)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|   1304|   2.5|      29|        10|     2006|       23|         16|         56|
|    75|   2011|   2.0|      29|        10|     2006|       23|         16|         39|
|    75|    420|   2.0|      29|        10|     2006|       23|         16|         42|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    832|   4.5|      29|        10|     2006|       23|         17|         49|
|    75|   2700|   4.5|      29|        10|     2006|       23|         17|         52|
|    75|   1485|   4.0|      29|        10|     2006|       23|         17|         46|
|    75|   1374|   4.0|      29|        10|     2006|       23|         17|         20|
|    75|    173|   3.5|      29|

### groupBy

Когда мы выполняем `groupBy()` в PySpark DataFrame, он возвращает объект `GroupedData`, который содержит следующие агрегатные функции:

`min()`, `max()`, `mean()`, `count()`, `sum()`, `avg()`, `agg()`, `pivot()` 

In [112]:
df.groupby('date_year').mean('rating').collect()

[Row(date_year=2003, avg(rating)=3.4279154852282083),
 Row(date_year=2007, avg(rating)=3.4367384383270894),
 Row(date_year=2006, avg(rating)=3.4294610550371356),
 Row(date_year=1997, avg(rating)=3.571018651362984),
 Row(date_year=2004, avg(rating)=3.3916603359500295),
 Row(date_year=1998, avg(rating)=3.5080568720379146),
 Row(date_year=2009, avg(rating)=3.369243156199678),
 Row(date_year=2001, avg(rating)=3.4490124195720484),
 Row(date_year=2005, avg(rating)=3.395790002433071),
 Row(date_year=2000, avg(rating)=3.5767733835530446),
 Row(date_year=2008, avg(rating)=3.486752231109465),
 Row(date_year=1999, avg(rating)=3.4925797284496367),
 Row(date_year=2002, avg(rating)=3.4140127388535033)]

мы уже умеем применять разные методы

In [113]:
df.groupby('date_year')\
  .mean('rating')\
  .sort('date_year')\
  .collect()

[Row(date_year=1997, avg(rating)=3.571018651362984),
 Row(date_year=1998, avg(rating)=3.5080568720379146),
 Row(date_year=1999, avg(rating)=3.4925797284496367),
 Row(date_year=2000, avg(rating)=3.5767733835530446),
 Row(date_year=2001, avg(rating)=3.4490124195720484),
 Row(date_year=2002, avg(rating)=3.4140127388535033),
 Row(date_year=2003, avg(rating)=3.4279154852282083),
 Row(date_year=2004, avg(rating)=3.3916603359500295),
 Row(date_year=2005, avg(rating)=3.395790002433071),
 Row(date_year=2006, avg(rating)=3.4294610550371356),
 Row(date_year=2007, avg(rating)=3.4367384383270894),
 Row(date_year=2008, avg(rating)=3.486752231109465),
 Row(date_year=2009, avg(rating)=3.369243156199678)]

In [114]:
df.filter(df.rating <= 2)\
  .groupby('date_year')\
  .count()\
  .withColumnRenamed('count', 'number')\
  .sort('date_year')\
  .collect()

[Row(date_year=1997, number=98),
 Row(date_year=1998, number=321),
 Row(date_year=1999, number=2270),
 Row(date_year=2000, number=4732),
 Row(date_year=2001, number=4793),
 Row(date_year=2002, number=5791),
 Row(date_year=2003, number=7595),
 Row(date_year=2004, number=10205),
 Row(date_year=2005, number=15982),
 Row(date_year=2006, number=23463),
 Row(date_year=2007, number=19792),
 Row(date_year=2008, number=15128),
 Row(date_year=2009, number=369)]

несколько колонок

In [115]:
df.groupBy('date_year', 'date_month') \
  .mean('rating', 'userID') \
  .sort('date_year', 'date_month') \
  .show()

+---------+----------+------------------+------------------+
|date_year|date_month|       avg(rating)|       avg(userID)|
+---------+----------+------------------+------------------+
|     1997|         9|3.7094972067039107| 35144.61452513967|
|     1997|        10|3.7762237762237763|34880.769230769234|
|     1997|        11| 3.360655737704918| 9451.180327868853|
|     1997|        12| 3.549618320610687| 20251.01526717557|
|     1998|         1|          3.765625|      22209.515625|
|     1998|         2|               3.3|           31662.4|
|     1998|         3| 3.238095238095238| 19582.04761904762|
|     1998|         4| 3.908333333333333|31813.866666666665|
|     1998|         5| 3.652054794520548|20393.520547945205|
|     1998|         6|             3.225|         27092.625|
|     1998|         7|3.5547550432276656|38376.249279538904|
|     1998|         8| 3.642857142857143|41151.357142857145|
|     1998|         9|3.1881533101045294| 46104.74912891986|
|     1998|        10|3.

Для того, чтобы делать несколько разных агрегаций и еще менять сразу имя столбца нужно немного изменить синтаксис

И стоит помнить, что персентили тут считаются приближенно

In [116]:
df.groupBy('date_year') \
    .agg(F.min('rating').alias('min_rating'), \
         F.mean('rating').alias('mean_rating'), \
         F.max('rating').alias('max_rating'),
         F.percentile_approx('rating', 0.5).alias('median')
         ) \
    .show()

+---------+----------+------------------+----------+------+
|date_year|min_rating|       mean_rating|max_rating|median|
+---------+----------+------------------+----------+------+
|     2003|       0.5|3.4279154852282083|       5.0|   3.5|
|     2007|       0.5|3.4367384383270894|       5.0|   3.5|
|     2006|       0.5|3.4294610550371356|       5.0|   3.5|
|     1997|       1.0| 3.571018651362984|       5.0|   4.0|
|     2004|       0.5|3.3916603359500295|       5.0|   3.5|
|     1998|       1.0|3.5080568720379146|       5.0|   4.0|
|     2009|       0.5| 3.369243156199678|       5.0|   3.5|
|     2001|       1.0|3.4490124195720484|       5.0|   4.0|
|     2005|       0.5| 3.395790002433071|       5.0|   3.5|
|     2000|       1.0|3.5767733835530446|       5.0|   4.0|
|     2008|       0.5| 3.486752231109465|       5.0|   3.5|
|     1999|       1.0|3.4925797284496367|       5.0|   4.0|
|     2002|       1.0|3.4140127388535033|       5.0|   4.0|
+---------+----------+------------------

Еще можно сделать pivot

In [117]:
df.groupBy('date_year')\
  .pivot('date_month')\
  .mean('rating')\
  .show(5)

+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|date_year|                 1|                 2|                 3|                 4|                 5|                 6|                 7|                 8|                 9|                10|                11|                12|
+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|     2003|3.4720922980979108| 3.333741687084354|3.4834824501032347|3.4579510703363914| 3.366686911630732|  3.55358230160341|3.5085277150739906| 3.447642637512812| 3.579355840124175| 3.165437788018433| 3.446676231468197| 3.440287081339713|
|     2007| 3.520191839855425| 3.3341516

### join

Куда же без них. Что есть:
* INNER
* LEFT OUTER
* RIGHT OUTER
* LEFT ANTI
* LEFT SEMI
* CROSS
* SELF JOIN

Благодаря оптимизации в датафреймах уже все хорошо работает, спасибо catalist, но чудеса не вечны и плохой код/незнание данных все равно даст о себе знать

Сделаем для себя несколько таблиц, чтобы можно было экспериментировать

In [118]:
df_mean_user_rating = df.groupBy('userID')\
                        .mean('rating')\
                        .withColumnRenamed('avg(rating)', 'avg_rating_all')

df_mean_user_rating_year = df.groupby('userID', 'date_year')\
                             .mean('rating')\
                             .withColumnRenamed('avg(rating)', 'avg_rating_year')

In [119]:
df_mean_user_rating.printSchema()

df_mean_user_rating_year.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- avg_rating_all: double (nullable = true)

root
 |-- userID: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- avg_rating_year: double (nullable = true)



И давайте все в 1 блоке кода, чтобы не растягивать

In [120]:
try:
    df.join(df_mean_user_rating, on=df.userID == df_mean_user_rating.userID, how='inner')\
      .join(df_mean_user_rating_year, on=[df.userID == df_mean_user_rating_year.userID,
                                          df.date_year == df_mean_user_rating_year.date_year],
            how='inner')
except Exception as e:
    print(e)

[AMBIGUOUS_REFERENCE] Reference `userID` is ambiguous, could be: [`userID`, `userID`].


Надо удалить дублирующиеся столбцы

In [121]:
res_join = df.alias('t').join(df_mean_user_rating.alias('t1'), on=F.col('t.userID') == F.col('t1.userID'), how='inner')\
  .drop(F.col('t1.userID'))\
  .join(df_mean_user_rating_year.alias('t2'), on=[F.col('t.userID') == F.col('t2.userID'),
                                      F.col('t.date_year') == F.col('t2.date_year')],
        how='inner')\
  .drop(F.col('t2.userID'))\
  .drop(F.col('t2.date_year'))

In [122]:
res_join.show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|    avg_rating_all|   avg_rating_year|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
| 16574|   4878|   4.5|      12|         2|     2006|       13|         35|          8| 4.033333333333333| 4.033333333333333|
| 14570|    377|   2.0|      19|         8|     2007|       10|         54|         35|2.7909407665505226|3.0208333333333335|
| 14570|    223|   4.0|       2|         7|     2003|        1|         41|         15|2.7909407665505226| 2.634715025906736|
| 14570|  30810|   4.0|      24|         9|     2005|       10|         28|         46|2.7909407665505226|3.4027777777777777|
| 18979|   1372|   1.5|      18|        11|     2007|       10|         31|         58|3.6904761904761907|            

### union и unionAll

Используются для объединения датафреймов с одинаковой структурой, используется `union`, так как `unionAll` с версии 2.0.0 более не используется

In [123]:
df1 = df.filter(df.date_year == 2006)
df2 = df.filter(df.date_year != 2006)

In [124]:
union_df = df1.union(df2)

print(df.count(), union_df.count())

855598 855598


**Desclaimer:** все по sql, надеюсь, помнят разницу между `union` и `union all`, когда union убирает дубликаты. Так вот pyspark ничего не удаляет, убрать дубликаты можно только через `drop_duplicates`, `distinct`

Также при `union` pyspark делает объединение по столбцам as is, не пытаясь понять, что в одном датафрейме нужный стобец на 1 позиции, а в другом он на 5. Для этого с версии 3.1 есть замечательный метод `unionByName`

### UDF - user defined functions

из курса про rdd помним про map, тут тоже можно перегнать все в rdd и делать map, но можно и через udf. Стоит отметить, что при этом мы теряем возможность оптимизации и произодительность в dataframe, так как udf - black box для спарка.

Но зато эти функции переиспользуемы и их можно применять в sql запросах, как те же udf в oracle

In [125]:
def udf_example(rating):
    rating = rating * 20
    return rating

In [126]:
from pyspark.sql.functions import udf

In [127]:
my_udf = udf(lambda x: udf_example(x), DoubleType())

In [128]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', my_udf(F.col('rating')))\
  .show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
|  3190|   6303|   3.5|      70.0|
|  6082|    858|   5.0|     100.0|
|  1724|   2181|   2.0|      40.0|
| 10905|  60126|   3.5|      70.0|
| 11162|   2393|   4.0|      80.0|
+------+-------+------+----------+
only showing top 5 rows



Для тех, кто любит декораторы

In [129]:
@udf(returnType=DoubleType()) 
def udf_example_decorator(rating):
    rating = rating * 20
    return rating

In [130]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', udf_example_decorator(F.col('rating')))\
  .show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
|  3190|   6303|   3.5|      70.0|
|  6082|    858|   5.0|     100.0|
|  1724|   2181|   2.0|      40.0|
| 10905|  60126|   3.5|      70.0|
| 11162|   2393|   4.0|      80.0|
+------+-------+------+----------+
only showing top 5 rows



Зарегистрируем функцию для будущих примеров с sql

In [131]:
spark.udf.register('udf_example_decorator', udf_example_decorator)

### SQL

Ну раз уж пошла такая тема, давайте рассмотрим, как можно сделать все при помощи любимого SQL

можно делать TempView и GlodalTempView, отличие в том, что обычный view будет жить, пока жива сессия спрака, а глобальная, пока жив sparkcontext

In [132]:
df.createOrReplaceTempView('df')

In [133]:
query = '''
select userID, movieID, rating, udf_example_decorator(rating) as rating_100
from
df
'''
spark.sql(query).show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
|  3190|   6303|   3.5|      70.0|
|  6082|    858|   5.0|     100.0|
|  1724|   2181|   2.0|      40.0|
| 10905|  60126|   3.5|      70.0|
| 11162|   2393|   4.0|      80.0|
+------+-------+------+----------+
only showing top 5 rows



Ну и наш join

In [134]:
df_mean_user_rating.createOrReplaceTempView('df_mean_user_rating')
df_mean_user_rating_year.createOrReplaceTempView('df_mean_user_rating_year')

In [135]:
query = '''
select t.*, t1.avg_rating_all, t2.avg_rating_year
from
df t, df_mean_user_rating t1, df_mean_user_rating_year t2
where
    t.userID = t1.userID and
    t.userID = t2.userID and
    t.date_year = t2.date_year
'''
spark.sql(query).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|    avg_rating_all|   avg_rating_year|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
| 16574|   4878|   4.5|      12|         2|     2006|       13|         35|          8| 4.033333333333333| 4.033333333333333|
| 14570|    377|   2.0|      19|         8|     2007|       10|         54|         35|2.7909407665505226|3.0208333333333335|
| 14570|    223|   4.0|       2|         7|     2003|        1|         41|         15|2.7909407665505226| 2.634715025906736|
| 14570|  30810|   4.0|      24|         9|     2005|       10|         28|         46|2.7909407665505226|3.4027777777777777|
| 18979|   1372|   1.5|      18|        11|     2007|       10|         31|         58|3.6904761904761907|            

### fill и fillna

Оба метода идентичны, заполняют пропуски

In [136]:
import numpy as np

In [137]:
data2 = [(2,'Michael Rose'),(3,'Robert Williams'),
     (4,'Rames Rose'),(5, None), (6, None),
     (None, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

А где пропуски?

In [138]:
df2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+----+
| id|name|
+---+----+
|  1|   2|
+---+----+



In [139]:
df2.show()

+----+---------------+
|  id|           name|
+----+---------------+
|   2|   Michael Rose|
|   3|Robert Williams|
|   4|     Rames Rose|
|   5|           NULL|
|   6|           NULL|
|NULL|        Fred Tf|
+----+---------------+



In [140]:
df2.fillna({'id': 0}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|           NULL|
|  6|           NULL|
|  0|        Fred Tf|
+---+---------------+



In [141]:
df2.fillna({'id': 0, 'name': 'Unknown'}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|        Unknown|
|  6|        Unknown|
|  0|        Fred Tf|
+---+---------------+



Аналогично

In [142]:
df2.na.fill({'id': 0, 'name': 'Unknown'}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|        Unknown|
|  6|        Unknown|
|  0|        Fred Tf|
+---+---------------+



Немного поговорим о том, как делаеть подвыборки

### sample и sampleBy

Не забываем про возможную некоторую недетерминированность

In [143]:
import pyspark.sql.functions as F

In [144]:
trans_data = spark.read.parquet('./spark_transactions.parquet')
trans_data = trans_data.withColumn('target', F.when(F.col('IsFraud')=='No', 0).otherwise(1))

In [145]:
trans_data.count()

24386900

In [146]:
trans_data.select(F.mean(F.col('target'))).show()

+--------------------+
|         avg(target)|
+--------------------+
|0.001220204290008...|
+--------------------+



In [147]:
trans_data.select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+--------+
|target|   count|
+------+--------+
|     1|   29757|
|     0|24357143|
+------+--------+



In [148]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.1, seed=3)
print(trans_data_simple.count())
trans_data_simple.select(F.mean(F.col('target'))).show()

2435202
+--------------------+
|         avg(target)|
+--------------------+
|0.001233983874848986|
+--------------------+



Стратификация

Тут важно понимать, что это не scikit-learn и стратификация предполагает, что вы по какому-то полю можете выбрать определенную долю наблюдений по его значениям

In [149]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-----+
|target|count|
+------+-----+
|     1|29757|
+------+-----+



Не указали какой-то ключ - его доля будет 0

In [150]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0, 0: 0.1}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-------+
|target|  count|
+------+-------+
|     1|  29757|
|     0|2436313|
+------+-------+



### Оконные функции

Оконные функции **F.func().over(Window.partitionBy().orderBy())**

In [151]:
from pyspark.sql.window import Window

In [152]:
trans_data_simple.select('User', 'Card', 'Year', 'Month', 'Day')\
.withColumn('rn_1', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc())))\
.withColumn('rn_2', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc(), F.col('Month').desc())))\
.withColumn('mean', F.mean('Day').over(Window.partitionBy('User', 'Card', 'Month')))\
.withColumn('lag', F.lag('Day').over(Window.partitionBy('User', 'Card', 'Month').orderBy(F.col('Year').asc(),
                                                                                         F.col('Month').desc(),
                                                                                         F.col('Day').desc())))\
.show()

+----+----+----+-----+---+----+----+------------------+----+
|User|Card|Year|Month|Day|rn_1|rn_2|              mean| lag|
+----+----+----+-----+---+----+----+------------------+----+
|   1|   3|2011|    1|  8|   9|  20|16.166666666666668|NULL|
|   1|   3|2012|    1| 28|  21|  30|16.166666666666668|   8|
|   1|   3|2013|    1| 28|  32|  47|16.166666666666668|  28|
|   1|   3|2013|    1|  1|  31|  46|16.166666666666668|  28|
|   1|   3|2014|    1| 30|  48|  58|16.166666666666668|   1|
|   1|   3|2016|    1|  2|  71|  80|16.166666666666668|  30|
|   1|   3|2011|    2| 23|  11|  19|            19.625|NULL|
|   1|   3|2011|    2| 21|  10|  18|            19.625|  23|
|   1|   3|2013|    2| 21|  34|  45|            19.625|  21|
|   1|   3|2013|    2| 19|  33|  44|            19.625|  21|
|   1|   3|2015|    2| 11|  59|  70|            19.625|  19|
|   1|   3|2017|    2| 15|  81|  92|            19.625|  11|
|   1|   3|2019|    2| 26| 103| 112|            19.625|  15|
|   1|   3|2019|    2| 2

### Сборка последовательностей

Сборка последовательностей через **collect_list()**

In [153]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)
 |-- target: integer (nullable = false)



In [154]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'Day').alias('sequence'))

In [155]:
trans_data_simple_seq.show()

+----+----+-------------+
|User|Card|     sequence|
+----+----+-------------+
|   0|   0| {$134.09, 1}|
|   0|   0|  {$53.91, 5}|
|   0|   0|  {$65.50, 9}|
|   0|   0| {$144.90, 9}|
|   0|   0|{$134.89, 23}|
|   0|   0|{$100.00, 25}|
|   0|   0| {$57.64, 27}|
|   0|   0|{$142.06, 10}|
|   0|   0|{$207.00, 14}|
|   0|   0| {$99.09, 15}|
|   0|   0| {$35.82, 25}|
|   0|   0|{$216.47, 26}|
|   0|   0| {$35.92, 30}|
|   0|   0|  {$42.01, 8}|
|   0|   0|{$151.49, 14}|
|   0|   0|{$113.32, 14}|
|   0|   0|{$127.33, 18}|
|   0|   0|{$228.92, 23}|
|   0|   0|{$148.42, 27}|
|   0|   0| {$210.96, 8}|
+----+----+-------------+
only showing top 20 rows



In [156]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('sequence').alias('sequence'))

In [157]:
trans_data_simple_seq.show(truncate=50)

+----+----+--------------------------------------------------+
|User|Card|                                          sequence|
+----+----+--------------------------------------------------+
|   1|   3|[{$15.61, 1}, {$51.90, 8}, {$85.96, 16}, {$62.8...|
|   3|   0|[{$87.52, 15}, {$398.49, 16}, {$135.19, 19}, {$...|
|   4|   0|[{$16.52, 1}, {$44.15, 5}, {$82.86, 8}, {$92.88...|
|  13|   1|[{$43.04, 18}, {$100.00, 19}, {$100.00, 19}, {$...|
|  16|   4|[{$39.82, 2}, {$57.01, 4}, {$71.12, 24}, {$35.6...|
|  19|   2|[{$27.33, 2}, {$18.92, 2}, {$33.42, 8}, {$44.64...|
|  27|   0|[{$5.27, 7}, {$125.02, 7}, {$4.11, 9}, {$9.48, ...|
|  29|   3|[{$22.45, 8}, {$2.82, 12}, {$109.43, 13}, {$2.9...|
|  31|   0|[{$13.13, 2}, {$7.21, 17}, {$5.80, 18}, {$8.22,...|
|  33|   0|[{$978.44, 1}, {$1.69, 4}, {$1.61, 21}, {$160.0...|
|  36|   0|[{$0.40, 7}, {$9.50, 14}, {$120.00, 23}, {$0.29...|
|  46|   0|[{$31.44, 1}, {$34.95, 2}, {$36.23, 26}, {$80.0...|
|  49|   0|[{$10.83, 14}, {$6.49, 19}, {$0.91, 3}, {$35

А что с этим делать дальше? Можно перейти к RDD и построчно обрабатывать

In [158]:
row = trans_data_simple_seq.rdd.take(1)

In [159]:
row

[Row(User=1, Card=3, sequence=[Row(Amount='$15.61', Day=1), Row(Amount='$51.90', Day=8), Row(Amount='$85.96', Day=16), Row(Amount='$62.83', Day=27), Row(Amount='$16.73', Day=20), Row(Amount='$3.91', Day=17), Row(Amount='$80.00', Day=10), Row(Amount='$24.61', Day=18), Row(Amount='$0.40', Day=8), Row(Amount='$172.53', Day=21), Row(Amount='$17.54', Day=23), Row(Amount='$69.50', Day=24), Row(Amount='$17.40', Day=24), Row(Amount='$15.99', Day=10), Row(Amount='$96.37', Day=3), Row(Amount='$17.83', Day=10), Row(Amount='$0.65', Day=16), Row(Amount='$14.95', Day=30), Row(Amount='$62.98', Day=10), Row(Amount='$78.70', Day=27), Row(Amount='$29.71', Day=28), Row(Amount='$89.99', Day=23), Row(Amount='$19.44', Day=11), Row(Amount='$61.89', Day=29), Row(Amount='$16.52', Day=21), Row(Amount='$18.14', Day=24), Row(Amount='$62.72', Day=17), Row(Amount='$107.61', Day=17), Row(Amount='$95.00', Day=17), Row(Amount='$-95.00', Day=17), Row(Amount='$97.27', Day=1), Row(Amount='$26.49', Day=28), Row(Amount='$2

In [160]:
row[0]['sequence']

[Row(Amount='$15.61', Day=1),
 Row(Amount='$51.90', Day=8),
 Row(Amount='$85.96', Day=16),
 Row(Amount='$62.83', Day=27),
 Row(Amount='$16.73', Day=20),
 Row(Amount='$3.91', Day=17),
 Row(Amount='$80.00', Day=10),
 Row(Amount='$24.61', Day=18),
 Row(Amount='$0.40', Day=8),
 Row(Amount='$172.53', Day=21),
 Row(Amount='$17.54', Day=23),
 Row(Amount='$69.50', Day=24),
 Row(Amount='$17.40', Day=24),
 Row(Amount='$15.99', Day=10),
 Row(Amount='$96.37', Day=3),
 Row(Amount='$17.83', Day=10),
 Row(Amount='$0.65', Day=16),
 Row(Amount='$14.95', Day=30),
 Row(Amount='$62.98', Day=10),
 Row(Amount='$78.70', Day=27),
 Row(Amount='$29.71', Day=28),
 Row(Amount='$89.99', Day=23),
 Row(Amount='$19.44', Day=11),
 Row(Amount='$61.89', Day=29),
 Row(Amount='$16.52', Day=21),
 Row(Amount='$18.14', Day=24),
 Row(Amount='$62.72', Day=17),
 Row(Amount='$107.61', Day=17),
 Row(Amount='$95.00', Day=17),
 Row(Amount='$-95.00', Day=17),
 Row(Amount='$97.27', Day=1),
 Row(Amount='$26.49', Day=28),
 Row(Amount='

# Домашнее задание

Куда же без домашки, верно?

Есть данные по транзакциям клиентов, ваша задача состоит в анализе этих данных и подготовки к структуре, которая похожа на ту структуру, которая сейчас часто нами используется при построении моделей на транзакциях + промежуточные задания.

Не забудьте делать всякие show после каждого задания, чтобы было видно результат

**Файл spark_transactions.parquet можете забрать в папке с записями лекций**

**Важно**
В домашнем задании старайтесь использовать максимально dataframe api, а не sql запросы.

## Данные

In [161]:
trans_data = spark.read.parquet('./spark_transactions.parquet')

In [162]:
trans_data.count()

24386900

In [163]:
trans_data.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 5 rows



Схема данных

In [164]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)



## Среднее количество транзакций у пользователя

In [165]:
trans_data.select(F.mean('User').alias('mean_user_trans')).show()

+------------------+
|   mean_user_trans|
+------------------+
|1001.0193350938414|
+------------------+



## Среднее количество карт у пользователя

In [166]:
trans_data.select('User', 'Card')\
          .distinct()\
          .select(F.mean('Card').alias('mean_user_card'))\
          .show()

+------------------+
|    mean_user_card|
+------------------+
|1.4720638540478905|
+------------------+



## Предобработка данных

Немного обработаем данные: Amount в float, из Time вытянем час транзакции и удалим исходный Time, Zip  к типу int

In [167]:
trans_data.show(3)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 3 rows



In [168]:
trans_data = trans_data.withColumn(
    'Amount', F.regexp_extract('Amount', r'\d+\.\d+(?:\.\d+)?', 0).cast('float'))\
                       .withColumn('Hour', F.hour('Time'))\
                       .withColumn('Minute', F.minute('Time'))\
                       .withColumn('Zip', F.col('Zip').cast('integer'))\
                       .drop('Time')

In [169]:
trans_data.show(3)

+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+----+------+
|User|Card|Year|Month|Day|Amount|          UseChip| MerchantCity|MerchantState|  Zip| MCC|IsFraud|Hour|Minute|
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+----+------+
|   0|   0|2002|    9|  1|134.09|Swipe Transaction|     La Verne|           CA|91750|5300|     No|   6|    21|
|   0|   0|2002|    9|  1| 38.48|Swipe Transaction|Monterey Park|           CA|91754|5411|     No|   6|    42|
|   0|   0|2002|    9|  2|120.34|Swipe Transaction|Monterey Park|           CA|91754|5411|     No|   6|    22|
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+----+------+
only showing top 3 rows



In [170]:
display(trans_data.dtypes)

[('User', 'bigint'),
 ('Card', 'bigint'),
 ('Year', 'bigint'),
 ('Month', 'bigint'),
 ('Day', 'bigint'),
 ('Amount', 'float'),
 ('UseChip', 'string'),
 ('MerchantCity', 'string'),
 ('MerchantState', 'string'),
 ('Zip', 'int'),
 ('MCC', 'bigint'),
 ('IsFraud', 'string'),
 ('Hour', 'int'),
 ('Minute', 'int')]

## Количество транзакций по годам

Посчитайте количество транзакций по годам, учитывая только те транзакции, объем которых был больше 100

In [171]:
trans_data.filter(F.col('Amount') >= 100)\
          .groupBy('Year')\
          .count()\
          .orderBy('Year')\
          .show(n=30)

+----+------+
|Year| count|
+----+------+
|1991|   341|
|1992|  1218|
|1993|  1639|
|1994|  2195|
|1995|  2853|
|1996|  3959|
|1997|  6724|
|1998| 10618|
|1999| 16116|
|2000| 24207|
|2001| 35099|
|2002| 47663|
|2003| 63374|
|2004| 81136|
|2005|101705|
|2006|122186|
|2007|140863|
|2008|159085|
|2009|173619|
|2010|190408|
|2011|198661|
|2012|201125|
|2013|205754|
|2014|206629|
|2015|208457|
|2016|210013|
|2017|210154|
|2018|209575|
|2019|209429|
|2020| 40096|
+----+------+



## Проверка пустых значений

Определите, есть ли пропуски в данных по каждому столбцу

In [172]:
def count_missing(df):
    return df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns])

In [173]:
%%time
count_missing(trans_data).show()

+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----+------+
|User|Card|Year|Month|Day|Amount|UseChip|MerchantCity|MerchantState|    Zip|MCC|IsFraud|Hour|Minute|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----+------+
|   0|   0|   0|    0|  0|     0|      0|           0|      2720821|2878135|  0|      0|   0|     0|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----+------+

CPU times: total: 78.1 ms
Wall time: 4min 9s


## Заполнение пустых значений

Заполните пропуски исходя из типа данных

In [174]:
display(trans_data.dtypes)

[('User', 'bigint'),
 ('Card', 'bigint'),
 ('Year', 'bigint'),
 ('Month', 'bigint'),
 ('Day', 'bigint'),
 ('Amount', 'float'),
 ('UseChip', 'string'),
 ('MerchantCity', 'string'),
 ('MerchantState', 'string'),
 ('Zip', 'int'),
 ('MCC', 'bigint'),
 ('IsFraud', 'string'),
 ('Hour', 'int'),
 ('Minute', 'int')]

In [175]:
trans_data = trans_data.fillna({
    'MerchantState': 'Unknown',
    'Zip': -1
})

In [176]:
%%time
count_missing(trans_data).show()

+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+----+------+
|User|Card|Year|Month|Day|Amount|UseChip|MerchantCity|MerchantState|Zip|MCC|IsFraud|Hour|Minute|
+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+----+------+
|   0|   0|   0|    0|  0|     0|      0|           0|            0|  0|  0|      0|   0|     0|
+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+----+------+

CPU times: total: 78.1 ms
Wall time: 6min 20s


## Статистики с использованием оконных функций

При помощи оконных функций для каждого клиента рассчитайте средний размер транзакции, количество транзакций и последнюю по дате транзакцию.

In [177]:
# Добавим поле с датой и временем транзакции:
trans_data = trans_data.withColumn(
    'TransDate',
    F.to_timestamp(
        F.concat(
            F.col('Year'), F.lit('-'), F.col('Month'), F.lit('-'), F.col('Day'),
            F.lit(' '),
            F.col('Hour'), F.lit(':'), F.col('Minute')
        )
    )
)

In [178]:
# Рассчитываем статистики с помощью оконных функций:
w = Window.partitionBy('User')
user_stats = trans_data.select(
    'User',
    F.avg('Amount').over(w).alias('mean_trans_amount'),
    F.count('*').over(w).alias('trans_count'),
    F.max('TransDate').over(w).alias('last_TransDate')
)

In [179]:
%%time
user_stats.show(5)

+----+-----------------+-----------+-------------------+
|User|mean_trans_amount|trans_count|     last_TransDate|
+----+-----------------+-----------+-------------------+
|   0|90.17621047155345|      19963|2020-02-28 07:36:00|
|   0|90.17621047155345|      19963|2020-02-28 07:36:00|
|   0|90.17621047155345|      19963|2020-02-28 07:36:00|
|   0|90.17621047155345|      19963|2020-02-28 07:36:00|
|   0|90.17621047155345|      19963|2020-02-28 07:36:00|
+----+-----------------+-----------+-------------------+
only showing top 5 rows

CPU times: total: 15.6 ms
Wall time: 2min 7s


## Изменение структуры

Теперь самое время сгруппировать данные по каждому клиенту (можно использовать collect_list для сбора данных после агрегации)
Когда будете делать агрегацию, то возьмите только часть выборки, например, через sample, для всей выборки либо не хватит памяти, либо очень долго считать

In [180]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.001, seed=777)

print(trans_data_simple.count())
trans_data_simple.show(5)

24349
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+----+------+-------------------+
|User|Card|Year|Month|Day|Amount|          UseChip| MerchantCity|MerchantState|  Zip| MCC|IsFraud|Hour|Minute|          TransDate|
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+----+------+-------------------+
|   0|   0|2002|    9| 11|  99.0|Swipe Transaction|    Mira Loma|           CA|91752|5541|     No|  13|    23|2002-09-11 13:23:00|
|   0|   0|2006|    9|  2| 61.89|Swipe Transaction|Monterey Park|           CA|91755|5942|     No|  13|    39|2006-09-02 13:39:00|
|   0|   0|2010|    9|  3|109.28|Swipe Transaction|     La Verne|           CA|91750|5912|     No|   6|    18|2010-09-03 06:18:00|
|   0|   0|2013|    1| 21|114.79|Swipe Transaction|     La Verne|           CA|91750|5311|     No|   6|     3|2013-01-21 06:03:00|
|   0|   0|2014|    5|  5| 77.67|Swipe Transaction|     La Verne|           C

In [181]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'TransDate', 'MCC').alias('Transactions'))\
                                         .select("*").orderBy(F.rand(seed=777)) # Тут шафлим, иначе и так все упорядочено
trans_data_simple_seq.show(5)

+----+----+--------------------+
|User|Card|        Transactions|
+----+----+--------------------+
|1528|   0|{138.24, 2002-08-...|
| 842|   2|{89.62, 2020-02-0...|
|1868|   0|{70.45, 2016-05-0...|
|1704|   0|{57.28, 2014-01-0...|
|1616|   0|{8.96, 2016-06-03...|
+----+----+--------------------+
only showing top 5 rows



In [182]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('Transactions').alias('Transactions'))
trans_data_simple_seq.show(5, truncate=120)

+----+----+------------------------------------------------------------------------------------------------------------------------+
|User|Card|                                                                                                            Transactions|
+----+----+------------------------------------------------------------------------------------------------------------------------+
|   0|   0|[{77.67, 2014-05-05 06:28:00, 5912}, {61.89, 2006-09-02 13:39:00, 5942}, {114.79, 2013-01-21 06:03:00, 5311}, {99.0, ...|
|   0|   2|[{39.73, 2017-08-26 06:44:00, 5912}, {26.48, 2006-09-15 21:49:00, 7832}, {119.57, 2017-01-16 05:57:00, 3504}, {59.4, ...|
|   0|   3|[{138.13, 2014-05-11 06:13:00, 5411}, {66.5, 2016-09-09 09:48:00, 7538}, {51.89, 2011-04-05 06:56:00, 5411}, {58.51, ...|
|   1|   1|[{124.54, 2015-08-21 04:38:00, 4900}, {0.75, 2019-09-14 13:10:00, 5411}, {62.55, 2011-06-08 10:10:00, 5411}, {128.75,...|
|   1|   2|[{0.6, 2009-06-15 13:29:00, 5651}, {100.69, 2010-10-05 06:

## Обработка последовательностей

Напишите python функцию, которая возьмет данные после агрегации последовательностей, отсортирует их внутри по дате и времени и преобразует к формату python dict:
{'User': User,
'Card': Card,
'sequence':{
    'amount': [последовательность],
    'year': [последовательность],
    'month': [последовательность],
    'day': [последовательность],
    'time': [последовательность],
    'MCC': [последовательность]
}
}




Выведите как пример одну преобразованную запись, результаты сохраните на диск через rdd в pickle

In [183]:
def preprocess_trans(row):
    
    sorted_transactions = sorted(row.Transactions, key=lambda x: x.TransDate)
    
    amount_list = [transaction.Amount for transaction in sorted_transactions]
    date_list = [str(transaction.TransDate) for transaction in sorted_transactions]
    mcc_list = [transaction.MCC for transaction in sorted_transactions]

    return {
        'User': row.User,
        'Card': row.Card,
        'Transactions': {
            'Amount': amount_list,
            'TransDate': date_list,
            'MCC': mcc_list
        }
    }

In [184]:
trans_dict = trans_data_simple_seq.rdd.map(preprocess_trans)

In [185]:
trans_dict.collect()[0]

{'User': 0,
 'Card': 0,
 'Transactions': {'Amount': [99.0,
   61.88999938964844,
   109.27999877929688,
   114.79000091552734,
   77.66999816894531],
  'TransDate': ['2002-09-11 12:23:00',
   '2006-09-02 12:39:00',
   '2010-09-03 05:18:00',
   '2013-01-21 05:03:00',
   '2014-05-05 05:28:00'],
  'MCC': [5541, 5942, 5912, 5311, 5912]}}

In [186]:
trans_data_simple_seq.select("User", "Card", F.explode("Transactions"))\
                     .select("User", "Card", F.col("col.*"))\
                     .orderBy("User", "Card", "TransDate")\
                     .show()

+----+----+------+-------------------+----+
|User|Card|Amount|          TransDate| MCC|
+----+----+------+-------------------+----+
|   0|   0|  99.0|2002-09-11 13:23:00|5541|
|   0|   0| 61.89|2006-09-02 13:39:00|5942|
|   0|   0|109.28|2010-09-03 06:18:00|5912|
|   0|   0|114.79|2013-01-21 06:03:00|5311|
|   0|   0| 77.67|2014-05-05 06:28:00|5912|
|   0|   2| 26.48|2006-09-15 21:49:00|7832|
|   0|   2|  59.4|2009-05-13 13:33:00|5211|
|   0|   2| 11.38|2011-05-23 13:30:00|5812|
|   0|   2|119.57|2017-01-16 05:57:00|3504|
|   0|   2| 39.73|2017-08-26 06:44:00|5912|
|   0|   3|142.77|2003-09-16 06:20:00|5912|
|   0|   3| 80.37|2005-06-18 14:13:00|5719|
|   0|   3| 450.0|2005-06-30 06:36:00|4722|
|   0|   3| 51.89|2011-04-05 06:56:00|5411|
|   0|   3| 58.51|2011-08-22 13:36:00|5311|
|   0|   3| 83.93|2012-01-04 06:14:00|5300|
|   0|   3|139.41|2012-06-04 05:05:00|5411|
|   0|   3|138.13|2014-05-11 06:13:00|5411|
|   0|   3|125.54|2016-04-19 06:18:00|5311|
|   0|   3|  66.5|2016-09-09 09:

In [187]:
sc.stop()

In [188]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}